In [24]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# 소득과 나이, 그리고 이상치를 포함한 데이터 생성
data = {'income': [50000, 52000, 48000, 150000, 51000, 49000, 1000000],
        'age': [25, 30, 28, 45, 26, 27, 35]}
df = pd.DataFrame(data)

In [73]:
# 1. 스케일러 객체 생성
scaler_std = StandardScaler()

# 2. 데이터에 fit_transform 적용 (fit과 transform을 동시에)
df_std = scaler_std.fit_transform(df)

# 결과 확인 (Numpy 배열로 반환됨)
print("--- StandardScaler Result ---")
print(df_std)
print(f"\nMean: {df_std.mean(axis=0)}")
print(f"Std: {df_std.std(axis=0)}")

# Scikit-Learn을 사용 못할 경우 직접 연산 하는 방법
# ddof=0 으로 표준편차 설정을 해줘야 모수 표준편차로 계산됨 (기본 ddof=1은 표본 표준편차)
scaled_res = ((df - df.mean()) / df.std(ddof=0)).to_numpy()

# 일부 열만 스케일링하고 싶을 경우, cols를 지정한다.
cols = ['income', 'age']
scaled_res = ((df[cols] - df[cols].mean()) / df[cols].std(ddof=0)).to_numpy()

# Scikit-Learn과 동일한 결과를 볼 수 있다.
print(scaled_res)
print(f"\nMean: {scaled_res.mean(axis=0)}")
print(f"Std: {scaled_res.std(axis=0)}")

--- StandardScaler Result ---
[[-0.45673461 -0.89640241]
 [-0.45064482 -0.13118084]
 [-0.46282441 -0.43726947]
 [-0.15224487  2.16448387]
 [-0.45368972 -0.7433581 ]
 [-0.45977951 -0.59031378]
 [ 2.43591795  0.63404073]]

Mean: [ 0.00000000e+00 -9.51619735e-17]
Std: [1. 1.]
[[-0.45673461 -0.89640241]
 [-0.45064482 -0.13118084]
 [-0.46282441 -0.43726947]
 [-0.15224487  2.16448387]
 [-0.45368972 -0.7433581 ]
 [-0.45977951 -0.59031378]
 [ 2.43591795  0.63404073]]

Mean: [ 0.00000000e+00 -3.17206578e-17]
Std: [1. 1.]


In [74]:
# 1. 스케일러 객체 생성
scaler_minmax = MinMaxScaler()

# 2. 데이터에 fit_transform 적용
df_minmax = scaler_minmax.fit_transform(df)

# 결과 확인
print("--- MinMaxScaler Result ---")
print(df_minmax)
print(f"\nMin: {df_minmax.min(axis=0)}")
print(f"Max: {df_minmax.max(axis=0)}")

# Scikit-Learn을 사용 못할 경우 직접 연산 하는 방법
# ddof=0 으로 표준편차 설정을 해줘야 모수 표준편차로 계산됨 (기본 ddof=1은 표본 표준편차)
scaled_res = ((df - df.min()) / (df.max()-df.min())).to_numpy()

# 일부 열만 스케일링하고 싶을 경우, cols를 지정한다.
cols = ['income', 'age']
scaled_res = ((df[cols] - df[cols].min()) / (df[cols].max()-df[cols].min())).to_numpy()

# Scikit-Learn과 동일한 결과를 볼 수 있다.
print(scaled_res)
print(f"\nMin: {scaled_res.min(axis=0)}")
print(f"Max: {scaled_res.max(axis=0)}")

--- MinMaxScaler Result ---
[[0.00210084 0.        ]
 [0.00420168 0.25      ]
 [0.         0.15      ]
 [0.10714286 1.        ]
 [0.00315126 0.05      ]
 [0.00105042 0.1       ]
 [1.         0.5       ]]

Min: [0. 0.]
Max: [1. 1.]
[[0.00210084 0.        ]
 [0.00420168 0.25      ]
 [0.         0.15      ]
 [0.10714286 1.        ]
 [0.00315126 0.05      ]
 [0.00105042 0.1       ]
 [1.         0.5       ]]

Min: [0. 0.]
Max: [1. 1.]


In [77]:
# 1. 스케일러 객체 생성
scaler_robust = RobustScaler()

# 2. 데이터에 fit_transform 적용
df_robust = scaler_robust.fit_transform(df)

# 결과 확인
print("--- RobustScaler Result ---")
print(df_robust)

# Scikit-Learn을 사용 못할 경우 직접 연산 하는 방법

# 중앙값과 3분위수-1분위수를 통해 IQR 값을 구하고 진행
median = df.median()
iqr = df.quantile(0.75) - df.quantile(0.25)
scaled_res = ((df - median) / iqr).to_numpy()

# 일부 열만 스케일링하고 싶을 경우, cols를 지정한다.
cols = ['income', 'age']
scaled_res = ((df[cols] - df[cols].median()) / (df[cols].quantile(0.75)-df[cols].quantile(0.25))).to_numpy()

# Scikit-Learn과 동일한 결과를 볼 수 있다.
print(scaled_res)

--- RobustScaler Result ---
[[-0.01941748 -0.5       ]
 [ 0.01941748  0.33333333]
 [-0.05825243  0.        ]
 [ 1.9223301   2.83333333]
 [ 0.         -0.33333333]
 [-0.03883495 -0.16666667]
 [18.42718447  1.16666667]]
[[-0.01941748 -0.5       ]
 [ 0.01941748  0.33333333]
 [-0.05825243  0.        ]
 [ 1.9223301   2.83333333]
 [ 0.         -0.33333333]
 [-0.03883495 -0.16666667]
 [18.42718447  1.16666667]]


In [79]:
df

,income,age
0,50000,25
1,52000,30
2,48000,28
3,150000,45
4,51000,26
5,49000,27
6,1000000,35


In [81]:
pd.DataFrame(scaler_robust.inverse_transform(df_robust)).astype("int")

,0,1
0,50000,25
1,52000,30
2,48000,28
3,150000,45
4,51000,26
5,49000,27
6,1000000,35
